# Small Language Model from LLM
Download an LLM, prune and quantize it, and benchmark it each step of the way.

## Start by downloading an LLM
I was going to use Llama 2 just because of how ubiquitous it currently is. However, I realized it requires HuggingFace authentication, because of how Meta AI has an approval process. To avoid cluttering the code with authentication, I just went with Mistral AI's Mistral model instead. We could choose larger versions of this model. However, to prove out and practice these model-optimization concepts, we can iterate faster with a smaller model like 7B.

According to a discussion on HuggingFace, Llama-2 7B requires 28GB of GPU RAM. Assuming it is similar for Mistral 7B, and to be on the safe side, I'll over-provision with an ml.g5.4xlarge for my SageMaker Studio Notebook.

### Set up environment
At first I got the error `KeyError: 'mistral'` when running `from_pretrained()`
The solution was on [the model's HuggingFace page](https://huggingface.co/mistralai/Mistral-7B-v0.1#troubleshooting)

In [2]:
!pip install --upgrade transformers

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_repo = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForCausalLM.from_pretrained(model_repo, torch_dtype=torch.float16).to("cuda")

2024-03-30 16:29:06.163052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]